In [1]:
#필요 라이브러리 호출
import csv
import numpy as np
import pandas as pd
from datetime import datetime
from konlpy.tag import Kkma
from collections import Counter

In [2]:
# 파일 저장 함수
def save_csv_dict(dicts,filename):
    with open("excel/"+filename+".csv", 'w') as file:
        writer = csv.writer(file)
        for k, v in dicts.items():
            writer.writerow([k, v])

# 상품 구매와 비교적 관계가 없는 단어들 제거 함수
except_word=['광고','현금','유저','정도','유료','피파온라인','PC','질문','지금','낙찰','게임','생각','가치','얼마','사람','정보통','대리','이번','후기','이유','요즘', '감사', '시간', '가능', '시간', '진짜', '이상']
def count_sort(counter_list,top):
    counts={}
    i=1
    for key, count in dict(sorted(counter_list.items(), key=lambda item: item[1], reverse=True)).items():
        if i <= top and key not in except_word:
            counts[key]=count
            i+=1
        elif i>top:
            break

    return counts

In [4]:
#단어 추출
file = open("lists/fmko_list.txt", "r", encoding='utf-8')
fmko_posts = file.readlines()
file.close()

posts=[]
for i in range(0,len(fmko_posts)-1,2):
    tmp=fmko_posts[i].replace('\n','').split('//구분자//')
    if '피파온라인X' not in tmp[0]:
        tmp[1]=datetime.strptime(tmp[1],'%Y.%m.%d %H:%M')
        tmp[3:6]=list(map(int,tmp[3:6]))
        tmp.append(fmko_posts[i+1].replace('\n','').split('//구분자//'))
        posts.append(tmp)

posts=sorted(posts,key=lambda x:x[1],reverse=True)

posts1=[]
comments=[]
for post in posts:
    posts1.append(post[:6])
    comments.append(post[-1])

post1=np.array(posts1)
main, date, post, view_num, recommend_num, comment_num=post1[:, 0], post1[:, 1], post1[:, 2], post1[:, 3], post1[:, 4], post1[:, 5]

In [8]:
# 게시글 제목, 내용, 댓글별 형태소 분석을 통한 '명사' 및 '외국어' 추출
kkma=Kkma()

main_noun=[]
for main_tmp in main:
    try:
        pos=kkma.pos(main_tmp)
        noun=[txt[0] for txt in pos if txt[1] in ['OL','NNG'] and (len(txt[0])>=2 or txt[0] in ('대','중','소'))]
        main_noun.append(noun)
    except:''
all_main_noun=sum(main_noun,[])

post_noun=[]
for post_tmp in post:
    try:
        pos=kkma.pos(post_tmp)
        noun=[txt[0] for txt in pos if txt[1] in ['OL','NNG'] and (len(txt[0])>=2 or txt[0] in ('대','중','소'))]
        post_noun.append(noun)
    except: post_noun.append([""])
all_post_noun=sum(post_noun,[])

comments_noun=[]
for i,comments_tmp in enumerate(comments):
    comment_noun=[]
    for comment in comments_tmp:
        try:
            pos=kkma.pos(comment)
            noun=[txt[0] for txt in pos if txt[1] in ['OL','NNG'] and (len(txt[0])>=2 or txt[0] in ('대','중','소'))]
            comment_noun.append(noun)
        except:
            comment_noun.append([""])
    comments_noun.append(sum(comment_noun,[]))
all_comments_noun=sum(comments_noun,[])

In [12]:
# 게시글 제목, 내용, 댓글별 단어 출현 횟수 파악 및 모든 게시글의 단어 출현 횟수 파악
main_freq=Counter(all_main_noun)
post_freq=Counter(all_post_noun)
comment_freq=Counter(all_comments_noun)
all_freq=Counter(sum([all_main_noun,all_post_noun,all_comments_noun],[]))

In [14]:
# 상품 구매와 연관된 단어들의 출현 횟수 추출 및 csv파일 저장
main_count=count_sort(main_freq,20)
post_count=count_sort(post_freq,20)
comment_count=count_sort(comment_freq,20)

save_csv_dict(main_count,'main_count')
save_csv_dict(post_count,'post_count')
save_csv_dict(comment_count,'comment_count')

In [16]:
# 날짜별 게시글 수 파악
date_notime=[dt.date() for dt in date]
date_count=Counter(date_notime)
save_csv_dict(date_count,'date_count')

In [41]:
# 게시글 제목, 내용, 댓글별 추출한 주요 단어 중 형태소 분석에서 분리된 단어 결합 및 상품구매와 관련성이 떨어지는 단어 제거
except_word=['제가','FMKOREA','RE','아이','오늘','일차','현재','HTTPS','COM','WWW','자체','있음','느낌','기준','이제','본인','경우','포함','최소','유료광고','현금','유저','정도','피파온라인','PC','질문','지금','대리낙찰','게임','생각','가치','얼마','사람','정보통','이번','후기','이유','요즘', '감사', '시간', '가능', '시간', '진짜', '이상']

main_r=[" ".join(noun) for noun in main_noun]
post_r=[" ".join(noun) for noun in post_noun]
comment_r=[" ".join(noun) for noun in comments_noun]

main_r=[noun.replace("감독 모드",'감독모드') for noun in main_r]
post_r=[noun.replace("감독 모드",'감독모드') for noun in post_r]
comment_r=[noun.replace("감독 모드",'감독모드') for noun in comment_r]

main_r=[noun.replace("이적 시장",'이적시장') for noun in main_r]
post_r=[noun.replace("이적 시장",'이적시장') for noun in post_r]
comment_r=[noun.replace("이적 시장",'이적시장') for noun in comment_r]

main_r=[noun.replace("구단 가치",'구단가치') for noun in main_r]
post_r=[noun.replace("구단 가치",'구단가치') for noun in post_r]
comment_r=[noun.replace("구단 가치",'구단가치') for noun in comment_r]

main_r=[noun.replace("대리 낙찰",'대리낙찰') for noun in main_r]
post_r=[noun.replace("대리 낙찰",'대리낙찰') for noun in post_r]
comment_r=[noun.replace("대리 낙찰",'대리낙찰') for noun in comment_r]

main_r=[noun.replace("유료 광고",'유료광고') for noun in main_r]
post_r=[noun.replace("유료 광고",'유료광고') for noun in post_r]
comment_r=[noun.replace("유료 광고",'유료광고') for noun in comment_r]

main_r=[noun.replace("카드 결제",'카드결제') for noun in main_r]
post_r=[noun.replace("카드 결제",'카드결제') for noun in post_r]
comment_r=[noun.replace("카드 결제",'카드결제') for noun in comment_r]

main_r=[noun.replace("라이브 퍼포먼스",'라이브퍼포먼스') for noun in main_r]
post_r=[noun.replace("라이브 퍼포먼스",'라이브퍼포먼스') for noun in post_r]
comment_r=[noun.replace("라이브 퍼포먼스",'라이브퍼포먼스') for noun in comment_r]

main_r=[noun.replace("대리 업체",'대리업체') for noun in main_r]
post_r=[noun.replace("대리 업체",'대리업체') for noun in post_r]
comment_r=[noun.replace("대리 업체",'대리업체') for noun in comment_r]

main_r=[noun.replace("고객 센터",'고객센터') for noun in main_r]
post_r=[noun.replace("고객 센터",'고객센터') for noun in post_r]
comment_r=[noun.replace("고객 센터",'고객센터') for noun in comment_r]

main_r=[noun.replace("알파 거래",'알파거래') for noun in main_r]
post_r=[noun.replace("알파 거래",'알파거래') for noun in post_r]
comment_r=[noun.replace("알파 거래",'알파거래') for noun in comment_r]

main_r=[noun.replace("마일 스톤",'마일스톤') for noun in main_r]
post_r=[noun.replace("마일 스톤",'마일스톤') for noun in post_r]
comment_r=[noun.replace("마일 스톤",'마일스톤') for noun in comment_r]

for except_w in except_word:
    main_r=[noun.replace(except_w,'') for noun in main_r]
    post_r=[noun.replace(except_w,'') for noun in post_r]
    comment_r=[noun.replace(except_w,'') for noun in comment_r]

In [42]:
# 추출한 단어들, 날짜, 조회수, 추천수, 댓글수 저장
posts=pd.DataFrame(np.transpose(np.array([list(map(str,date_notime)),view_num,recommend_num, comment_num,main_r,post_r, comment_r])))
posts.to_csv("C:/Users/wjdgh/OneDrive/바탕 화면/과제 A/A-2/excel/posts.csv",header=False,index=False)